In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.pipeline import Pipeline
import pandas as pd

In [2]:

class ModelTrainer:
    def __init__(self):
        self.models = {}
        self.best_model = None
        self.best_score = 0
        self.results = {}

    def train_models(self, X_train, y_train, X_val, y_val):
        model_grid = {
            "LogisticRegression": {
                "model": LogisticRegression(max_iter=1000),
                "params": {
                    "C": [0.01, 0.1, 1, 10],
                    "penalty": ['l2'],
                    "solver": ['lbfgs']
                }
            },
            "RandomForest": {
                "model": RandomForestClassifier(random_state=42),
                "params": {
                    "n_estimators": [100, 200],
                    "max_depth": [None, 10, 20],
                    "min_samples_split": [2, 5]
                }
            },
            "GradientBoosting": {
                "model": GradientBoostingClassifier(random_state=42),
                "params": {
                    "n_estimators": [100, 200],
                    "learning_rate": [0.05, 0.1],
                    "max_depth": [3, 5]
                }
            },
            "XGBoost": {
                "model": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
                "params": {
                    "n_estimators": [100, 200],
                    "learning_rate": [0.05, 0.1],
                    "max_depth": [3, 5]
                }
            }
        }

        for name, config in model_grid.items():
            print(f"\n🔍 Training and tuning {name}...")
            grid_search = GridSearchCV(config['model'], config['params'], cv=3, scoring='f1', n_jobs=-1)
            grid_search.fit(X_train, y_train)

            best_model = grid_search.best_estimator_
            y_pred = best_model.predict(X_val)

            acc = accuracy_score(y_val, y_pred)
            f1 = f1_score(y_val, y_pred)

            self.results[name] = {
                "accuracy": acc,
                "f1_score": f1,
                "model": best_model,
                "classification_report": classification_report(y_val, y_pred, output_dict=True)
            }

            if f1 > self.best_score:
                self.best_score = f1
                self.best_model = best_model

    def get_best_model(self):
        return self.best_model

    def get_results(self):
        return pd.DataFrame({
            model: {
                "Accuracy": round(result["accuracy"], 4),
                "F1 Score": round(result["f1_score"], 4)
            }
            for model, result in self.results.items()
        }).T


In [3]:
from sklearn.model_selection import train_test_split

# 1. Load your cleaned data
df = pd.read_csv("../data/cleaned_churn_data.csv")  # replace with your actual filename

# 2. Separate features and target
X = df.drop(columns=['churn'])
y = df['churn']

# 3. Split into train/validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

trainer = ModelTrainer()
trainer.train_models(X_train, y_train, X_val, y_val)

print("📊 Performance Summary:")
print(trainer.get_results())

best_model = trainer.get_best_model()
y_pred = best_model.predict(X_val)
print("📄 Classification Report:")
print(classification_report(y_val, y_pred))
print("🏆 Best Model:", best_model.__class__.__name__)




🔍 Training and tuning LogisticRegression...

🔍 Training and tuning RandomForest...

🔍 Training and tuning GradientBoosting...

🔍 Training and tuning XGBoost...


c:\Users\ASUS\OneDrive\Desktop\customerChurn\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [22:57:02] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


📊 Performance Summary:
                    Accuracy  F1 Score
LogisticRegression    0.8061    0.6029
RandomForest          0.8107    0.6209
GradientBoosting      0.8154    0.6232
XGBoost               0.8224    0.6381
📄 Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.91      0.88       937
           1       0.71      0.58      0.64       347

    accuracy                           0.82      1284
   macro avg       0.78      0.75      0.76      1284
weighted avg       0.82      0.82      0.82      1284

🏆 Best Model: XGBClassifier


In [4]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = best_model.predict(X_val)
y_prob = best_model.predict_proba(X_val)[:,1]

print(classification_report(y_val, y_pred))
print("ROC AUC:", roc_auc_score(y_val, y_prob))


              precision    recall  f1-score   support

           0       0.85      0.91      0.88       937
           1       0.71      0.58      0.64       347

    accuracy                           0.82      1284
   macro avg       0.78      0.75      0.76      1284
weighted avg       0.82      0.82      0.82      1284

ROC AUC: 0.8617283684824032


In [5]:
import joblib

# Get churn probabilities using predict_proba (class 1 = churn)
y_proba = best_model.predict_proba(X_val)[:, 1]

# Optional: show few examples
prob_df = pd.DataFrame({
    "Actual": y_val.values,
    "Predicted": y_pred,
    "Churn_Probability": y_proba
})
print(prob_df.head())




   Actual  Predicted  Churn_Probability
0       0          0           0.007882
1       0          1           0.838810
2       1          1           0.677160
3       0          0           0.251933
4       0          0           0.129099


In [9]:
# Save model with churn probability support
joblib.dump(best_model, "../model/best_churn_model.pkl")
print("✅ Model saved with probability support")

✅ Model saved with probability support
